In [8]:
import numpy  as np
import torch
from pytorchcv.model_provider import get_model as ptcv_get_model
from torch.autograd import Variable
import torchvision.models as models
import matplotlib
import matplotlib.pyplot as plt
from os import makedirs

In [9]:
def splitTrainEvalSet(train,eva, path, shuffle = False, check_with_dummy_data=False):
    #load data and labels
    if check_with_dummy_data == False:
        data = np.loadtxt(path + '/data_train.txt')
        labels = np.loadtxt(path + '/labels_train.txt')
    else:
        data, labels = dummyArray()
    
    #combine data and labels to slice and shuffle
    temp = data.size//len(data)
    arrayLen,b = data.shape
    dataLabels = np.c_[data.reshape(len(data), -1), labels.reshape(len(labels), -1)]
    del labels
    del data
    
    #count label occurence to evenly split each label
    numOccurrenceEachLabel = []
    label = 0
    count = 0
    size = 0
    for i in range(arrayLen):
        if dataLabels[i][-1] == label:
            count += 1
        else:
            numOccurrenceEachLabel.append(count)
            size += int(count*train)
            count = 1
            label += 1
    numOccurrenceEachLabel.append(count)
    size += int(count*train)

    #slice and shuffle the data
    dataLabels_train = np.zeros((size,b+3))
    dataLabels_eval = np.zeros((arrayLen-size,b+3))
    occCount = 0
    countSteps1 = 0
    countSteps2 = 0
    for occ in numOccurrenceEachLabel:
        slice_ = dataLabels[occCount:occ+occCount]
        if shuffle == True:
            np.random.shuffle(slice_)
        occCount += occ
        step1 = int(occ*train)
        step2 = occ-step1
        dataLabels_train[countSteps1:countSteps1+step1] = slice_[:step1]
        dataLabels_eval[countSteps2:countSteps2+step2] = slice_[step1:]
        countSteps1 += step1
        countSteps2 += step2
    del dataLabels
    np.random.shuffle(dataLabels_train)
    np.random.shuffle(dataLabels_eval)
    
    #unravel data,labels array
    data_train_train = dataLabels_train[:, :temp].reshape(size,b)
    labels_train_train = dataLabels_train[:, temp:].reshape(size,3)
    data_train_eval = dataLabels_eval[:, :temp].reshape(arrayLen-size,b)
    labels_train_eval = dataLabels_eval[:, temp:].reshape(arrayLen-size,3)
    
    #save data and labels
    np.savetxt(path + '/data_train_train.txt', data_train_train, fmt='%d')
    np.savetxt(path + '/data_train_eval.txt', data_train_eval, fmt='%d')
    np.savetxt(path + '/labels_train_train.txt', labels_train_train)
    np.savetxt(path + '/labels_train_eval.txt', labels_train_eval)
    if check_with_dummy_data == True:
        return data_train_train, data_train_eval, labels_train_train, labels_train_eval
    else:
        #to Tensor
        data_train_train = torch.from_numpy(data_train_train)
        labels_train_train = torch.from_numpy(labels_train_train)
        data_train_eval = torch.from_numpy(data_train_eval)
        labels_train_eval = torch.from_numpy(labels_train_eval)
        #save as tensors
        torch.save(data_train_train, path + '/data_train_train.pt') 
        torch.save(data_train_eval, path + '/data_train_eval.pt')
        torch.save(labels_train_train, path + '/labels_train_train.pt') 
        torch.save(labels_train_eval, path + '/labels_train_eval.pt')
        return 0
    
def dummyArray():
    data = np.zeros((40,300))
    for i in range(data.shape[0]):
        data[i] = i
    labels = np.zeros((40,3))
    setLabels = [5,15,7,13]
    count = 0
    for i in range(len(setLabels)):
        for j in range(setLabels[i]):
            labels[count][2] = i
            count += 1
    return data,labels

def addKeypoints(sizeImageInterpolated, path, useTestset = False, saveImages = 0):
    #get model for the keypoint prediction
    net2 = models.detection.keypointrcnn_resnet50_fpn(pretrained=True, progress=True, num_classes=2, num_keypoints=17, pretrained_backbone=True)
    net2.eval()
    
    #load data
    if useTestset == False:
        data1 = torch.load(path + '/data_train_train.pt').float()
    else:
        data1 = torch.load(path + '/data_test.pt').float()
        
    #predict keypoints
    a,b = data1.shape
    keyArray = np.zeros((a, 17*3),dtype=float)
    print("start")
    for i, (datapoint) in enumerate(data1):
        if i%30 == 0:
            print("Train datapoint: " + str(i) + "/" + str(a) + " processed")
        x = datapoint.view(1,sizeImageInterpolated,sizeImageInterpolated,3)
        x = torch.transpose(x, 1, 3)
        x = torch.transpose(x, 2, 3)
        x /= 255
        predictions = net2(x)
        keypoints = predictions[0]['keypoints'].detach().numpy()
        if keypoints.shape[0] != 0:
            keyArray[i] = keypoints[0,:,0:3].flatten()
        
        #save every 2000 steps
        if i%2000 == 0:
            if useTestset == False:
                np.savetxt(path + '/data_train_train_key.txt', keyArray, fmt='%d')
            else:
                np.savetxt(path + '/data_test_key.txt', keyArray, fmt='%d')
    
    #save
    if useTestset == False:
        np.savetxt(path + '/data_train_train_key.txt', keyArray, fmt='%d')
    else:
        np.savetxt(path + '/data_test_key.txt', keyArray, fmt='%d')
    
    #print keypoints into the images
    if saveImages != 0:
        try:
            makedirs('./images_key')
        except Exception as e:
            None
        for i in range(saveImages):
            img_key = data1[i].numpy().reshape((sizeImageInterpolated,sizeImageInterpolated,3))
            keys = keyArray[i].reshape((17,3))
            for k in keys:
                j = int(k[0])
                l = int(k[1])
                img_key[l-1:l+1,j-1:j+1] = [1.,0.,0.]
            matplotlib.image.imsave(path + '/images_key/' + str(i) +'.png', img_key)
    del data1
    del keyArray
    
    
    #also get keypoints for the eval data
    if useTestset == False:
        data2 = torch.load(path + '/data_train_eval.pt').float()
        a,b = data2.shape
        keyArray2 = np.zeros((a, 17*3),dtype=float)
        for i, (datapoint) in enumerate(data2):
            if i%30 == 0:
                print("Eval datapoint: " + str(i) + "/" + str(a) + " processed")
            x = datapoint.view(1,sizeImageInterpolated,sizeImageInterpolated,3)
            x = torch.transpose(x, 1, 3)
            x = torch.transpose(x, 2, 3)
            x /= 255
            predictions = net2(x)
            keypoints = predictions[0]['keypoints'].detach().numpy()
            if keypoints.shape[0] != 0:
                keyArray2[i] = keypoints[0,:,0:3].flatten()
            if i%2000 == 0:
                np.savetxt(path + '/data_train_eval_key.txt', keyArray2, fmt='%d')
        np.savetxt(path + '/data_train_eval_key.txt', keyArray2, fmt='%d')
    return 0

def keysToTensor(path, useTestset = False):
    if useTestset == False:
        keys1 = np.loadtxt(path + '/data_train_train_key.txt')
        keys2 = np.loadtxt(path + '/data_train_eval_key.txt')

        keysTen1 = torch.from_numpy(keys1)
        keysTen2 = torch.from_numpy(keys2)

        torch.save(keysTen1, path + '/data_train_train_key.pt') 
        torch.save(keysTen2, path + '/data_train_eval_key.pt') 
    else:
        keys1 = np.loadtxt(path + '/data_test_key.txt')
        keysTen1 = torch.from_numpy(keys1)
        torch.save(keysTen1, path + '/data_test_key.pt')
    
    
def getKeypointsInFormOfImages(path, useTestset = False):
    #load keypoints
    if useTestset == False:
        a = torch.load(path + '/data_train_train_key.pt').float()
        b = torch.load(path + '/data_train_eval_key.pt').float()
        
        #generate image-like keypoints from the eval data
        eva = torch.zeros(b.shape[0],100,100)
        for y,(i) in enumerate(b):
            keys = i.view(17,3)
            for x,(k) in enumerate(keys):
                j = int(k[0])
                l = int(k[1])
                eva[y,l,j] = 1.0+x*1.0
        torch.save(eva, path + '/data_train_eval_key_1.pt')
    else:
        a = torch.load(path + '/data_test_key.pt').float()
        
    #generate image-like keypoints from the train or test data 
    train = torch.zeros(a.shape[0],100,100)
    for y,(i) in enumerate(a):
        keys = i.view(17,3)
        for x,(k) in enumerate(keys):
            j = int(k[0])
            l = int(k[1])
            train[y,l,j] = 1.0+x*1.0
    
    if useTestset == False:
        torch.save(train, path + '/data_train_train_key_1.pt')
    else:
        torch.save(train, path + '/data_test_key_1.pt')

In [10]:
#split train,eval data
splitTrainEvalSet(0.8,0.2, path = './allData', shuffle = True, check_with_dummy_data=False)

0

In [12]:
#process train,eval data
addKeypoints(100, path = './allData', saveImages = 400)
keysToTensor(path = './allData')
getKeypointsInFormOfImages(path = './allData', useTestset = False)

In [10]:
#process test data
addKeypoints(100, path = './allData', saveImages = 50, useTestset = True)
keysToTensor(path = './allData', useTestset = True)
getKeypointsInFormOfImages(path = './allData', useTestset = True)